# Print latex tables 

In [1]:
import os
import pandas as pd

In [7]:
# Read excel overview
path_results = os.path.join('..', 'output')
df_scores = pd.read_excel(os.path.join(path_results, 'scores_overview.xlsx'), index_col=[0,1])
df_scores

IOU    Dice     TPR     TNR  FNBGR_0  \
unet_256_aug_ce_2_00   0       0.8345  0.9000  0.9348  0.9795   0.0563   
                       1       0.8335  0.8913  0.8978  0.9869   0.0870   
                       2       0.8111  0.8756  0.8769  0.9859   0.0975   
                       3       0.8323  0.8876  0.8815  0.9896   0.0978   
                       4       0.8435  0.9046  0.8997  0.9883   0.0855   
...                               ...     ...     ...     ...      ...   
unetbn_nonaug_iou_3_00 4       0.8899  0.9371  0.9592  0.9949   0.0310   
                       mean    0.7768  0.8349  0.9474  0.9166   0.0393   
                       std     0.2093  0.1937  0.0256  0.1573   0.0184   
                       median  0.8814  0.9321  0.9368  0.9949   0.0478   
                       iqr     0.0171  0.0117  0.0317  0.0008   0.0232   

                               FNBGR_1  
unet_256_aug_ce_2_00   0        0.1234  
                       1        0.1462  
                       2        0.2070  
                       3        0.1833  
                       4        0.2365  
...                                ...  
unetbn_nonaug_iou_3_00 4        0.1098  
                       mean     0.1394  
                       std      0.0612  
                       median   0.1799  
                       iqr      0.0705  

[648 rows x 6 columns]

In [8]:
# make dict of dataframes separated by tile size and wear_mode
scores_dict = {}
res = {'scores_512':r'^(?!.*256).*', 
       'scores_256':r'256\w*',}

for k, v in res.items():
    df = df_scores.filter(regex=v, axis=0)
    # scores_dict[k] = df[df.index.get_level_values(1).isin(['mean', 'std'])]
    scores_dict[k] = df[df.index.get_level_values(1).isin(['median', 'iqr'])]

In [16]:
# Create data frames of format: mean (std)
df_scores_display = {}
for name, df in scores_dict.items():
    print(name)
    models = df.index.get_level_values(0).unique()
    dff = pd.DataFrame(index=models, columns=df.columns)
    for m in models:
        dff.loc[m,:] = df.loc[(m, 'median'),:].apply(lambda x: "{:.3f}".format(x)) + " (" + df.loc[(m,"iqr"),:].apply(lambda x: "{:.3f}".format(x)) + ")"
    dff['train data aug.'] = dff.index.str.split('_').str[-4]
    dff['train data aug.'] = dff['train data aug.'].str.replace(r'^aug$','full').replace('littleaug','moderate').replace('nonaug','none')
    #dff['mode'] = dff.index.str.split('_').str[-2]
    dff['mode'] = [m.replace('2', 'binary').replace('3', 'multi class') for m in dff.index.str.split('_').str[-2]]
    dff['BN'] = dff.index.str.contains('unetbn')
    dff.loc[dff['BN'], ['mode']] = dff['mode']+' + BN'
    dff = dff.drop('BN', axis=1)
    dff['loss'] = dff.index.str.split('_').str[-3].str.upper().str.replace('IOU', 'IoU')
    
    #df_scores_display[name] = dff.sort_values(by=['mode', 'bn', 'loss', 'aug'])
    
    df_scores_display[name] = dff.set_index(['mode', 'loss', 'train data aug.']).sort_index()
    display(df_scores_display[name])

scores_512


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_2712\190882831.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  dff['train data aug.'] = dff['train data aug.'].str.replace(r'^aug$','full').replace('littleaug','moderate').replace('nonaug','none')


IOU           Dice  \
mode             loss train data aug.                                 
binary           CE   full             0.811 (0.071)  0.875 (0.050)   
                      moderate         0.863 (0.013)  0.920 (0.016)   
                      none             0.832 (0.005)  0.893 (0.011)   
                 FCE  full             0.747 (0.018)  0.828 (0.015)   
                      moderate         0.862 (0.016)  0.920 (0.011)   
                      none             0.837 (0.013)  0.902 (0.008)   
                 IoU  full             0.835 (0.023)  0.892 (0.011)   
                      moderate         0.875 (0.009)  0.925 (0.006)   
                      none             0.861 (0.005)  0.917 (0.003)   
binary + BN      CE   full             0.844 (0.010)  0.902 (0.012)   
                      moderate         0.883 (0.008)  0.930 (0.015)   
                      none             0.860 (0.016)  0.911 (0.013)   
                 FCE  full             0.838 (0.013)  0.895 (0.010)   
                      moderate         0.864 (0.007)  0.911 (0.008)   
                      none             0.861 (0.011)  0.917 (0.008)   
                 IoU  full             0.858 (0.002)  0.916 (0.002)   
                      moderate         0.888 (0.006)  0.935 (0.005)   
                      none             0.878 (0.002)  0.929 (0.002)   
multi class      CE   full             0.823 (0.014)  0.887 (0.009)   
                      moderate         0.864 (0.013)  0.919 (0.008)   
                      none             0.854 (0.018)  0.909 (0.021)   
                 FCE  full             0.826 (0.032)  0.888 (0.032)   
                      moderate         0.864 (0.020)  0.920 (0.015)   
                      none             0.850 (0.009)  0.912 (0.005)   
                 IoU  full             0.794 (0.049)  0.862 (0.040)   
                      moderate         0.885 (0.006)  0.933 (0.005)   
                      none             0.872 (0.001)  0.925 (0.002)   
multi class + BN CE   full             0.854 (0.017)  0.906 (0.012)   
                      moderate         0.875 (0.002)  0.918 (0.004)   
                      none             0.859 (0.023)  0.916 (0.023)   
                 FCE  full             0.851 (0.016)  0.902 (0.015)   
                      moderate         0.872 (0.004)  0.918 (0.000)   
                      none             0.872 (0.007)  0.926 (0.004)   
                 IoU  full             0.861 (0.009)  0.916 (0.009)   
                      moderate         0.892 (0.001)  0.938 (0.002)   
                      none             0.881 (0.017)  0.932 (0.012)   

                                                 TPR            TNR  \
mode             loss train data aug.                                 
binary           CE   full             0.894 (0.023)  0.992 (0.003)   
                      moderate         0.919 (0.016)  0.995 (0.001)   
                      none             0.919 (0.017)  0.992 (0.004)   
                 FCE  full             0.888 (0.040)  0.990 (0.003)   
                      moderate         0.930 (0.022)  0.995 (0.000)   
                      none             0.922 (0.040)  0.993 (0.001)   
                 IoU  full             0.934 (0.038)  0.994 (0.002)   
                      moderate         0.935 (0.021)  0.995 (0.000)   
                      none             0.926 (0.005)  0.995 (0.001)   
binary + BN      CE   full             0.899 (0.017)  0.995 (0.001)   
                      moderate         0.936 (0.010)  0.995 (0.000)   
                      none             0.921 (0.020)  0.996 (0.000)   
                 FCE  full             0.896 (0.026)  0.995 (0.000)   
                      moderate         0.928 (0.020)  0.995 (0.001)   
                      none             0.913 (0.012)  0.996 (0.001)   
                 IoU  full             0.933 (0.008)  0.994 (0.000)   
                      moderate         0.943 (0.010)  0.995 (0.001)   
 

scores_256


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_2712\190882831.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  dff['train data aug.'] = dff['train data aug.'].str.replace(r'^aug$','full').replace('littleaug','moderate').replace('nonaug','none')


IOU           Dice  \
mode             loss train data aug.                                 
binary           CE   full             0.834 (0.002)  0.891 (0.012)   
                      moderate         0.893 (0.003)  0.938 (0.002)   
                      none             0.871 (0.008)  0.926 (0.007)   
                 FCE  full             0.823 (0.018)  0.885 (0.010)   
                      moderate         0.882 (0.010)  0.933 (0.007)   
                      none             0.869 (0.011)  0.925 (0.008)   
                 IoU  full             0.854 (0.013)  0.909 (0.014)   
                      moderate         0.896 (0.004)  0.942 (0.003)   
                      none             0.887 (0.010)  0.935 (0.008)   
binary + BN      CE   full             0.884 (0.007)  0.931 (0.009)   
                      moderate         0.905 (0.006)  0.946 (0.006)   
                      none             0.885 (0.001)  0.933 (0.002)   
                 FCE  full             0.869 (0.016)  0.926 (0.009)   
                      moderate         0.908 (0.012)  0.950 (0.013)   
                      none             0.891 (0.004)  0.938 (0.003)   
                 IoU  full             0.900 (0.005)  0.945 (0.002)   
                      moderate         0.909 (0.001)  0.951 (0.001)   
                      none             0.896 (0.022)  0.941 (0.019)   
multi class      CE   full             0.846 (0.006)  0.900 (0.011)   
                      moderate         0.895 (0.016)  0.942 (0.011)   
                      none             0.874 (0.014)  0.923 (0.014)   
                 FCE  full             0.836 (0.010)  0.896 (0.009)   
                      moderate         0.893 (0.003)  0.939 (0.001)   
                      none             0.870 (0.013)  0.922 (0.012)   
                 IoU  full             0.866 (0.008)  0.920 (0.005)   
                      moderate         0.902 (0.011)  0.946 (0.007)   
                      none             0.888 (0.008)  0.937 (0.006)   
multi class + BN CE   full             0.891 (0.003)  0.938 (0.001)   
                      moderate         0.909 (0.010)  0.950 (0.007)   
                      none             0.892 (0.011)  0.937 (0.008)   
                 FCE  full             0.882 (0.020)  0.933 (0.012)   
                      moderate         0.906 (0.005)  0.947 (0.005)   
                      none             0.885 (0.006)  0.935 (0.003)   
                 IoU  full             0.888 (0.010)  0.937 (0.007)   
                      moderate         0.912 (0.003)  0.952 (0.002)   
                      none             0.893 (0.014)  0.940 (0.009)   

                                                 TPR            TNR  \
mode             loss train data aug.                                 
binary           CE   full             0.898 (0.018)  0.987 (0.002)   
                      moderate         0.938 (0.006)  0.988 (0.001)   
                      none             0.941 (0.015)  0.985 (0.002)   
                 FCE  full             0.901 (0.021)  0.983 (0.004)   
                      moderate         0.942 (0.005)  0.986 (0.002)   
                      none             0.936 (0.005)  0.983 (0.001)   
                 IoU  full             0.929 (0.004)  0.984 (0.001)   
                      moderate         0.953 (0.007)  0.986 (0.002)   
                      none             0.939 (0.012)  0.986 (0.001)   
binary + BN      CE   full             0.947 (0.007)  0.986 (0.003)   
                      moderate         0.946 (0.013)  0.990 (0.001)   
                      none             0.930 (0.027)  0.988 (0.003)   
                 FCE  full             0.911 (0.020)  0.990 (0.001)   
                      moderate         0.942 (0.010)  0.990 (0.001)   
                      none             0.941 (0.004)  0.987 (0.001)   
                 IoU  full             0.951 (0.006)  0.987 (0.002)   
                      moderate         0.953 (0.006)  0.988 (0.001)   
 

In [17]:
# Create latex code for tables
for name, df in df_scores_display.items():
    #df.index = df.index.str.replace(r'_2_\d*', '', regex=True).str.replace(r'_3_\d*', '', regex=True)
    #df.index = df.index.str.replace('_', '\_')
    latex = df.to_latex(index=True, float_format="%.2f", multirow=True, caption=name.replace('_','\_'), column_format=3*'l'+len(df.columns)*'c', escape=False)
    print(latex)

\begin{table}
\centering
\caption{scores\_512}
\begin{tabular}{lllcccccc}
\toprule
                 &     &      &            IOU &           Dice &            TPR &            TNR &        FNBGR_0 &        FNBGR_1 \\
mode & loss & train data aug. &                &                &                &                &                &                \\
\midrule
\multirow{9}{*}{binary} & \multirow{3}{*}{CE} & full &  0.811 (0.071) &  0.875 (0.050) &  0.894 (0.023) &  0.992 (0.003) &  0.093 (0.037) &  0.219 (0.070) \\
                 &     & moderate &  0.863 (0.013) &  0.920 (0.016) &  0.919 (0.016) &  0.995 (0.001) &  0.068 (0.021) &  0.175 (0.030) \\
                 &     & none &  0.832 (0.005) &  0.893 (0.011) &  0.919 (0.017) &  0.992 (0.004) &  0.068 (0.012) &  0.160 (0.043) \\
\cline{2-9}
                 & \multirow{3}{*}{FCE} & full &  0.747 (0.018) &  0.828 (0.015) &  0.888 (0.040) &  0.990 (0.003) &  0.090 (0.043) &  0.197 (0.098) \\
                 &     & moderate &  0.862

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_2712\4004744408.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=True, float_format="%.2f", multirow=True, caption=name.replace('_','\_'), column_format=3*'l'+len(df.columns)*'c', escape=False)
